In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

import BFast
import matplotlib
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams.update({'font.size': 13})
tf.config.list_physical_devices('GPU')

ModuleNotFoundError: jax requires jaxlib to be installed. See https://github.com/google/jax#installation for installation instructions.

In [ ]:
BoxSize = 1000.
kF = 2*np.pi/BoxSize
grid = 256

In [ ]:
df = np.load(f"df_m_256_PCS_z=0.npy")

In [ ]:
help(BFast.Bk_fast)

In [ ]:
%time Bks_fast = BFast.Bk_fast(df,BoxSize,3.,3.,27,'All','PCS',verbose=True)

In [ ]:
%time Bks = BFast.Bk(df,BoxSize,3.,3.,27,'All','PCS',verbose=True)

In [ ]:
plt.semilogy(Bks_fast[:,-2])
plt.semilogy(Bks[:,-2])

In [ ]:
Bks_fast[:,-2] - Bks[:,-2]